# Step 8 재귀에서 반복문으로

## 8.1 현재의 Variable 클래스

In [5]:
import numpy as np

In [6]:
class Variable:
  '''
    - 이 variable을 만들어 준 함수(func : creator)를 지정해줌
    -> define-by-run을 위해서 이 variable이 생성(forward)됨과 동시에
       backward를 위한 func를 연결해주는 역할을 함.
  '''
  def __init__(self, data):
    self.data = data
    self.grad = None
    self.creator = None
  
  def set_creator(self, func):
    self.creator = func

  def backward(self):
    '''
      - 앞에 있는 변수들 grad를 재귀적으로 구해보자
    '''
    f = self.creator  # 현재 변수의 creator를 불러옴
    if f is not None: 
      x = f.input     # creator의 input을 가져온다
                      # 이미 다 연결이 되어있기 때문에 가져올 수 있음
      x.grad = f.backward(self.grad)  # creator의 backward를 통해 
                                      # creator의 input의 grad를 구할 수 있음
      x.backward()   # 이 동작을 재귀적으로 수행

In [7]:
# 반복문으로 새로 구현
class Variable:
  '''
    - 이 variable을 만들어 준 함수(func : creator)를 지정해줌
    -> define-by-run을 위해서 이 variable이 생성(forward)됨과 동시에
       backward를 위한 func를 연결해주는 역할을 함.
  '''
  def __init__(self, data):
    self.data = data
    self.grad = None
    self.creator = None
  
  def set_creator(self, func):
    self.creator = func

  def backward(self):
    '''
      - 재귀가 아닌 반복문으로 구현한 backward
    '''
    funcs = [self.creator]     # creator list 생성
    while funcs :              # funcs의 원소가 없어질 때까지 반복
      f = funcs.pop()          # funcs의 가장 끝 함수를 pop
      x, y = f.input, f.output # 해당 함수의 input, output을 꺼냄
      x.grad = f.backward(y.grad) # 해당 함수의 input의 gradient를 구함

      if x.creator is not None:
        funcs.append(x.creator) # creator input x가 creator func를 가지고 있으면
                                # func list에 추가하고 해당 동작 반복. 

'''
 - 반복문으로 구현시, 복잡한 계산 그래프를 다룰 때 확장성이 뛰어나다.
 - 사실 직관적으로 봐도 그렇다. 
   variable의 creator가 반드시 하나일 이유가 없기 때문에, 
   여러 creator들을 다루어주는 것이 필수적일 것이다.
'''

In [8]:
class Function:
  '''
    - output 저장
    - output이 생성됨과 동시에 creator와 연결
  '''
  def __call__(self, input):
    x = input.data
    y = self.forward(x)  
    output = Variable(y)

    output.set_creator(self) # 출력 변수가 생성됨과 동시에 creator를 설정(생성과 동시에 '연결')
    
    self.input = input
    self.output= output # 출력도 저장
    return output

  def forward(self, x):
    raise NotImplementedError()

  def backward(self, gy):
    raise NotImplementedError()

class Square(Function):
  def forward(self, x):
    y = x ** 2
    return y

  def backward(self, gy):
    x = self.input.data
    gx = 2 * x * gy
    return gx

class Exp(Function):
  def forward(self, x):
    y = np.exp(x)
    return y

  def backward(self, gy):
    x = self.input.data
    gx = np.exp(x) * gy
    return gx

A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 역전파
y.grad = np.array(1.0)
y.backward()
print(x.grad) # 3.297442541400256

3.297442541400256
